# Business Understanding

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Understanding

In [2]:
df = pd.read_csv('../data/en-US.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222186 entries, 0 to 222185
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              222186 non-null  object 
 1   html                            222186 non-null  object 
 2   political                       222186 non-null  int64  
 3   not_political                   222186 non-null  int64  
 4   title                           222139 non-null  object 
 5   message                         222186 non-null  object 
 6   thumbnail                       222186 non-null  object 
 7   created_at                      222186 non-null  object 
 8   updated_at                      222186 non-null  object 
 9   lang                            222186 non-null  object 
 10  images                          222186 non-null  object 
 11  impressions                     222186 non-null  int64  
 12  political_probab

In [13]:
df.head()

,id,html,political,not_political,title,message,thumbnail,created_at,updated_at,lang,...,targets,advertiser,entities,page,lower_page,targetings,paid_for_by,targetedness,listbuilding_fundraising_proba,page_id
0,6163390790373,"<div class=""_5pa- userContentWrapper""><div cla...",0,0,The Daily Show,"<p>“The virus in Outbreak is so bad, all the s...",https://fbpac-ads-public.s3.amazonaws.com/v/t1...,2020-03-23 23:36:27.107353+00,2020-04-21 16:24:56.611341+00,en-US,...,"[{""target"": ""Interest"", ""segment"": ""Saturday N...",The Daily Show,[],https://www.facebook.com/thedailyshow/,https://www.facebook.com/thedailyshow/,"{""{\""data\"":{\""waist_targeting_data\"":[{\""__ty...",Comedy Central,0.0,NaN,7.976227e+09
1,6156647062563,"<div class=""_5pa- userContentWrapper""><div cla...",0,1,Ben Shapiro,<p>You're not powerless:</p>,https://fbpac-ads-public.s3.amazonaws.com/v/t1...,2020-03-23 21:48:47.600788+00,2020-04-21 16:24:35.402693+00,en-US,...,"[{""target"": ""Activity on the Facebook Family"",...",Ben Shapiro,[],https://www.facebook.com/officialbenshapiro/,https://www.facebook.com/officialbenshapiro/,"{""{\""data\"":{\""waist_targeting_data\"":[{\""__ty...",The Daily Wire,0.0,NaN,2.038051e+14
2,6177669705764,"<div class=""_5pa- userContentWrapper""><div cla...",0,1,FWD.us,<p>URGENT: Help families impacted by coronavir...,https://fbpac-ads-public.s3.amazonaws.com/v/t1...,2020-03-23 21:23:59.357889+00,2020-04-21 16:26:20.188464+00,en-US,...,"[{""target"": ""List"", ""segment"": """"}, {""target"":...",FWD.us,[],https://www.facebook.com/fwdus/,https://www.facebook.com/fwdus/,"{""{\""data\"":{\""waist_targeting_data\"":[{\""__ty...",FWD.us,0.0,NaN,2.941966e+14
3,23844182612400000,"<div class=""ego_unit"" id=""u_13_1""><div class=""...",0,0,Water.org,"<span class=""o_87ruke84q"">Access to safe water...",https://fbpac-ads-public.s3.amazonaws.com/v/t4...,2020-03-23 23:34:30.501902+00,2020-03-24 19:49:54.042532+00,en-US,...,"[{""target"": ""Retargeting"", ""segment"": ""people ...",NaN,[],NaN,NaN,"{""{\""data\"":{\""waist_targeting_data\"":[{\""__ty...",NaN,0.0,NaN,NaN
4,23844398507420386,"<div class=""ego_unit"" id=""u_5d_0""><div class=""...",0,1,Commercial and Residential,"<span class=""w_1km-lqpz-f"">Spring its just aro...",https://fbpac-ads-public.s3.amazonaws.com/v/t4...,2020-02-25 23:57:28.443817+00,2020-02-27 03:49:20.608505+00,en-US,...,"[{""target"": ""MinAge"", ""segment"": 18}, {""target...",NaN,[],NaN,NaN,"{""{\""data\"":{\""waist_targeting_data\"":[{\""__ty...",NaN,0.0,NaN,NaN


I Wanted to see how what exactly are in the columns that im unsure what they represent exactly so that i can create a new dataframe with only the columns i need.

In [14]:
# The Following are columns I have checked because i was unsure about them before deciding to drop/clean them 
# df['entities'].value_counts()
# df['targetedness'][5000]
# df['targetings'][5000]
# df['targeting'][5000]
# df['targets'][5000]
# df['lang'].value_counts()
# df['listbuilding_fundraising_proba'].value_counts()
# df['paid_for_by'].value_counts()

In [45]:
data = df.iloc[:, [4, 5, 7, 8, 11, 12, 15, 17, 23]]

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222186 entries, 0 to 222185
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   title                           222139 non-null  object 
 1   message                         222186 non-null  object 
 2   created_at                      222186 non-null  object 
 3   updated_at                      222186 non-null  object 
 4   impressions                     222186 non-null  int64  
 5   political_probability           222186 non-null  float64
 6   targets                         218324 non-null  object 
 7   entities                        219046 non-null  object 
 8   listbuilding_fundraising_proba  148125 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 15.3+ MB


In [47]:
data.head()

,title,message,created_at,updated_at,impressions,political_probability,targets,entities,listbuilding_fundraising_proba
0,The Daily Show,"<p>“The virus in Outbreak is so bad, all the s...",2020-03-23 23:36:27.107353+00,2020-04-21 16:24:56.611341+00,2,0.188052,"[{""target"": ""Interest"", ""segment"": ""Saturday N...",[],NaN
1,Ben Shapiro,<p>You're not powerless:</p>,2020-03-23 21:48:47.600788+00,2020-04-21 16:24:35.402693+00,1,0.978527,"[{""target"": ""Activity on the Facebook Family"",...",[],NaN
2,FWD.us,<p>URGENT: Help families impacted by coronavir...,2020-03-23 21:23:59.357889+00,2020-04-21 16:26:20.188464+00,2,0.999212,"[{""target"": ""List"", ""segment"": """"}, {""target"":...",[],NaN
3,Water.org,"<span class=""o_87ruke84q"">Access to safe water...",2020-03-23 23:34:30.501902+00,2020-03-24 19:49:54.042532+00,4,0.709614,"[{""target"": ""Retargeting"", ""segment"": ""people ...",[],NaN
4,Commercial and Residential,"<span class=""w_1km-lqpz-f"">Spring its just aro...",2020-02-25 23:57:28.443817+00,2020-02-27 03:49:20.608505+00,4,0.718452,"[{""target"": ""MinAge"", ""segment"": 18}, {""target...",[],NaN


In [48]:
dfclean = data.dropna()

In [49]:
dfclean = dfclean.replace(['[]'],'None')
dfclean

,title,message,created_at,updated_at,impressions,political_probability,targets,entities,listbuilding_fundraising_proba
7673,Beto O'Rourke,<p>Beto just announced he’s running for presid...,2019-03-17 03:37:40.808522+00,2020-04-19 23:17:05.135112+00,1,0.999999,None,"[{""entity"": ""Beto"", ""entity_type"": ""Region""}]",1.000000
7674,Planned Parenthood,"<p>Say it loud, say it proud: Our rights, our ...",2019-03-17 03:35:24.825047+00,2020-04-19 23:17:07.265429+00,1,0.999999,None,"[{""entity"": ""Planned Parenthood"", ""entity_type...",1.000005
7675,We Raise Foundation,"<p>Help free people from poverty, violence, an...",2018-12-23 22:01:42.175883+00,2020-04-17 23:26:40.084438+00,8,0.999137,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",None,0.999972
7676,Jobs for Minnesotans,<p>Minnesota deserves environmentally-friendly...,2019-03-17 02:05:42.53526+00,2020-04-19 23:17:13.884281+00,1,0.990709,None,"[{""entity"": ""Minnesota"", ""entity_type"": ""Regio...",0.503337
7678,Michael Negron,<p>I am proud to receive the endorsement of Al...,2019-03-17 01:34:35.150798+00,2020-04-19 23:17:17.357711+00,1,0.997485,None,"[{""entity"": ""Council"", ""entity_type"": ""Organiz...",0.281154
...,...,...,...,...,...,...,...,...,...
222112,SELC,"<p>Come out Tuesday, Jan. 15 and let DEQ know ...",2019-01-08 00:40:34.990806+00,2020-04-14 15:16:39.322225+00,2,0.991677,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""DEQ"", ""entity_type"": ""Organizatio...",0.573288
222119,We Raise Foundation,<p>Quality education provides vital opportunit...,2019-01-08 02:25:59.825005+00,2020-04-14 15:16:40.771441+00,2,0.998951,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",None,0.999920
222121,Human Rights Watch,<p>Make your New Year’s Resolution to defend h...,2019-01-01 23:09:25.140703+00,2020-04-14 15:16:42.187013+00,2,0.999876,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""New Year’s"", ""entity_type"": ""Even...",0.999798
222125,Planned Parenthood,<p>The Trump-Pence administration's next attac...,2019-01-01 19:17:25.945628+00,2020-04-14 15:16:43.158052+00,7,0.999998,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""Trump-Pence"", ""entity_type"": ""Org...",0.999973


In [50]:
dfclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142900 entries, 7673 to 222127
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   title                           142900 non-null  object 
 1   message                         142900 non-null  object 
 2   created_at                      142900 non-null  object 
 3   updated_at                      142900 non-null  object 
 4   impressions                     142900 non-null  int64  
 5   political_probability           142900 non-null  float64
 6   targets                         142900 non-null  object 
 7   entities                        142900 non-null  object 
 8   listbuilding_fundraising_proba  142900 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 10.9+ MB


In [38]:
dfclean = dfclean.replace(['[]'],'None')
dfclean

,title,message,created_at,updated_at,impressions,political_probability,targets,entities,listbuilding_fundraising_proba
7673,Beto O'Rourke,<p>Beto just announced he’s running for presid...,2019-03-17 03:37:40.808522+00,2020-04-19 23:17:05.135112+00,1,0.999999,None,"[{""entity"": ""Beto"", ""entity_type"": ""Region""}]",1.000000
7674,Planned Parenthood,"<p>Say it loud, say it proud: Our rights, our ...",2019-03-17 03:35:24.825047+00,2020-04-19 23:17:07.265429+00,1,0.999999,None,"[{""entity"": ""Planned Parenthood"", ""entity_type...",1.000005
7675,We Raise Foundation,"<p>Help free people from poverty, violence, an...",2018-12-23 22:01:42.175883+00,2020-04-17 23:26:40.084438+00,8,0.999137,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",None,0.999972
7676,Jobs for Minnesotans,<p>Minnesota deserves environmentally-friendly...,2019-03-17 02:05:42.53526+00,2020-04-19 23:17:13.884281+00,1,0.990709,None,"[{""entity"": ""Minnesota"", ""entity_type"": ""Regio...",0.503337
7678,Michael Negron,<p>I am proud to receive the endorsement of Al...,2019-03-17 01:34:35.150798+00,2020-04-19 23:17:17.357711+00,1,0.997485,None,"[{""entity"": ""Council"", ""entity_type"": ""Organiz...",0.281154
...,...,...,...,...,...,...,...,...,...
222112,SELC,"<p>Come out Tuesday, Jan. 15 and let DEQ know ...",2019-01-08 00:40:34.990806+00,2020-04-14 15:16:39.322225+00,2,0.991677,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""DEQ"", ""entity_type"": ""Organizatio...",0.573288
222119,We Raise Foundation,<p>Quality education provides vital opportunit...,2019-01-08 02:25:59.825005+00,2020-04-14 15:16:40.771441+00,2,0.998951,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",None,0.999920
222121,Human Rights Watch,<p>Make your New Year’s Resolution to defend h...,2019-01-01 23:09:25.140703+00,2020-04-14 15:16:42.187013+00,2,0.999876,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""New Year’s"", ""entity_type"": ""Even...",0.999798
222125,Planned Parenthood,<p>The Trump-Pence administration's next attac...,2019-01-01 19:17:25.945628+00,2020-04-14 15:16:43.158052+00,7,0.999998,"[{""target"": ""Age"", ""segment"": ""18 and older""},...","[{""entity"": ""Trump-Pence"", ""entity_type"": ""Org...",0.999973


In [59]:
dfclean['updated_at'].min()

'2017-09-07 16:20:33.833333+00'

In [57]:
dfclean['message'][82134]

'<p>Nine out of ten Americans want to protect endangered species and their habitat; however, congressional leaders have chosen to ignore this and continue to dismantle the ESA in favor of the fossil fuel industry. Where do you stand?</p>'

# Preprocessing